## Presentation

A notebook to process BCDI data using cdiutils package and PyNX phase retrieval

In [ ]:
import os
import hdf5plugin # if data are too large
from xrayutilities import en2lam

from cdiutils.processing.pipeline import BcdiPipeline
from cdiutils.load.authorized_keys import get_parameters_from_notebook_variables

## Processing parameters

In [ ]:
metadata = {
  'beamline_setup': 'ID01BLISS',
  'scan': 8,
  'sample_name': 'B18S2P1_0007',
  'experiment_file_path': "/data/visitor/hc4959/id01/20230214/raw/hc4959_id01.h5",
  'detector_data_path': None, # for ID01SPEC setup
  'edf_file_template': None, # for ID01SPEC setup
  'detector_name': 'mpx1x4',
  'flatfield_path': None,
  "alien_mask": None,
  'dump_dir': None,
  'reconstruction_file': "mode.h5"
}

metadata["dump_dir"] = (
    '/data/id01/inhouse/clatlan/experiments/hc4959/analysis'
    f'/results/{metadata["sample_name"]}/S{metadata["scan"]}/'
)

preprocessing_output_shape = [250, 250]
energy = 13000
roi = [0, 516, 0, 516]
hkl = [1, 1, 1]
det_reference_voxel_method = "max"
det_reference_voxel = None #[120, 327, 185]
binning_along_axis0 = None

det_calib_parameters = {
  'cch1': 207.46284910586556, # direct beam position vertical 
  'cch2': 69.95685534011558, # horizontal
  'pwidth1': 5.5e-05,
  'pwidth2': 5.5e-05,
  'distance': 1.2376291389386818,
  'tiltazimuth': 185.1428549886293,
  'tilt': 3.9999999483111504,
  'detrot': 0.3846153846153846,
  'outerangle_offset': 0.0
}

voxel_size = None
apodize = True
flip = False
isosurface = None

usetex = True
show = True
verbose = True
debug = True

# PyNX parameters
data = (
  metadata["dump_dir"]
  + f"S{metadata['scan']}_pynx_input_data.npz"
)

mask = (
  metadata["dump_dir"]
  + f"S{metadata['scan']}_pynx_input_mask.npz"
)

data2cxi = True
auto_center_resize = True

support_type = "square"
support_size = (40, 20, 40)
support = "auto"
support_threshold = "0.30, 0.40"
support_threshold_method = "rms"
support_only_shrink = False
support_update_period = 20
support_smooth_width_begin = 2
support_smooth_width_end = 1
support_post_expand = "1,-2,1"
psf = "pseudo-voigt,0.5,0.1,10"

nb_raar = 1000
nb_hio = 150
nb_er = 150
nb_ml = 10
nb_run = 15
nb_run_keep = 10

zero_mask = False
crop_output = 0
positivity = False
beta = 0.9
detwin = False

rebin = "1,1,1"
detector_distance = det_calib_parameters["distance"]
pixel_size_detector = 55e-6
wavelength = float(en2lam(energy) * 1e-10) # wavelength in meter

verbose = 100
output_format = "cxi"
live_plot = False
save_plot = True
mpi = "run"


# Load the parameters and parse them into the BcdiPipeline instance 
parameters = get_parameters_from_notebook_variables(dir(), globals())
bcdi_pipeline = BcdiPipeline(parameters=parameters)

### Preprocess the data: crop and center the data

In [ ]:
bcdi_pipeline.preprocess()

### Run PyNX phase retrieval using ESRF's p9 GPUs

In [ ]:
user = os.environ["USER"]
key_file_path = os.environ["HOME"] + "/.ssh/id_rsa"
number_of_nodes = 2

print(
    f"Will use the user name '{user}' "
    f"and the private key file path: '{key_file_path}'"
)

bcdi_pipeline.phase_retrieval(
    # machine="slurm-nice-devel",
    machine="lid01pwr9",
    user=user,
    number_of_nodes=number_of_nodes,
    key_file_path=key_file_path,
    remove_last_results=True
)

### Filter/sort the reconstructions and decompose them into one mode


In [ ]:
bcdi_pipeline.find_best_candidates(nb_to_keep=5)
bcdi_pipeline.mode_decomposition()

### Finally postprocess the data: orthogonalize, compute phase, dispacement, strain, dspacing...

In [ ]:
bcdi_pipeline.postprocess()